In [ ]:
# Import packages 
import sys
sys.path.append('../Plotting/')
""" Now you can imprt modules in ../Plotting"""
import xyp_plot as xyp

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

#Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


#Evaluation
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ana as a

import importlib


In [ ]:
# Constants
Pi=3.141592653589793
R_d = 287.0 # J K-1 kg-1
C_p = 1005.7 # J K-1 kg-1
grav = 9.8 # m s-2
kappa = R_d/C_p
print(kappa)

In [ ]:
# Specify directory for data
drct='/glade/p/cesm/amwg_dev/juliob/SAMwrf/Curtains/'
tag='ndg04'
fi=drct+'SAMwrf_'+tag+'_ML_super_v6.nc'
ds=xr.open_dataset( fi )


In [ ]:
MXD=ds['MXDIS']
CLN=ds['CLNGT']
ANG=ds['ANGLL']
SGH=ds['SGH']
ANX=ds['ANGLX']
print(np.shape(ANG.values.flatten()))
print(np.shape(ANX.values.flatten()))
plt.xlim(-360,360)
plt.ylim(-360,360)
#plt.plot(ANX.values.flatten(),'.')
#plt.plot(ANG.values.flatten(),'.')
plt.scatter(ANG.values.flatten(), ANX.values.flatten() )
# Better look at ANGLL in SE

In [ ]:
mxd=MXD.values
cln=CLN.values
ang=ANG.values
anx=ANX.values
sgh=SGH.values
print(mxd.shape)

In [ ]:
print(list(ds.variables))

Utn=ds['UTEND_NDG']*86400.
Vtn=ds['VTEND_NDG']*86400.
Utc=ds['UTEND_CORE']*86400.
Vtc=ds['VTEND_CORE']*86400.
Utgw=ds['UTEND_GWDTOT']*86400.
Vtgw=ds['VTEND_GWDTOT']*86400.
U=ds['U']
V=ds['V']
T=ds['T']

# Save-off dimensions of met-data
nT,nL,nS_0=np.shape(U)
print( nL,nT,nS_0 )

In [ ]:
plt.plot(T[1,:,2000])

In [ ]:
#calculate 3D pressure for
ps=ds['PS']
hyam=ds['hyam']
hybm=ds['hybm']
#plev=d4['lev']

gps=np.average( ps, axis=0 )
ghya=np.average( hyam , axis=0 )
ghyb=np.average( hybm , axis=0 )
p3=a.press(PS=ps,hybm=hybm,hyam=hyam )
print(np.shape(p3),np.shape(T))

In [ ]:
hybi=ds['hybi']
hyai=ds['hyai']

p3e=a.press(PS=ps,hybm=hybi,hyam=hyai )

# WARNING !!!!!

In [ ]:
# DO NOT DO THIS IDL-style CRAP !!!!! :
#   > theta=T.values
#   > te = T.values
#   > theta = calculations .... 
#
# 'te' is modified by this even though it is never on the LHS


In [ ]:
# Calculate Theta and N-squared (approximately) ...
# using a constant

theta=np.zeros( (nT, nL, nS_0 )    ) # T.values
te=T.values
plt.plot(te[1,:,2000])

print( id(te) )
print( id(theta) )

print(np.shape(hybm))
for iT in np.arange(nT):
    #p3=a.press(PS=ps[iT,:],hybm=hybm[iT,:],hyam=hyam[iT,:] )
    theta[iT,:,:] = (( 100_000. / p3[iT,:,:]  )**kappa ) * te[iT,:,:]
    #print( iT, end=',')
    
plt.plot(te[1,:,2000])

In [ ]:
rho_d = p3 / (R_d * te )
#p

In [ ]:
print( np.arange(nL) )
print(' .. ')
print( np.arange( nL-1,-1,-1))

In [ ]:
zhgte=np.zeros( (nT,nL+1,nS_0) )
for iL in np.arange(nL-1,-1,-1):
    zhgte[:,iL,:] = zhgte[:,iL+1,:] + \
                    ( p3e[:,iL+1,:]-p3e[:,iL,:]  )  \
                    /( grav * rho_d[:,iL,:] )
    print(iL,end=',')

    

In [ ]:
#te=T.values

zhgtm = 0.5 * (zhgte[:,0:nL,:]+zhgte[:,1:nL+1,:])
print(np.shape(zhgtm))

plt.plot( theta[1,:,2000] , zhgtm[1,:,2000] )


#print( rho_d[1,:,2000] )
#print( te[1,:,2000] )
#print( p3e[1,:,2000] )
#print( geophm[1,:,2000]  )



In [ ]:
plt.plot(rho_d[1,:,2000] )

## Begin setting up data for ML


## Here we isolate attention to regions with topography

In [ ]:
# Pick out grid cells with MXD[0,:] bigger than 50m.
# 
oo=np.where(mxd[0,:]>50.)
print(np.shape(oo))



mxd=mxd[0,oo[0][:]]
cln=cln[0,oo[0][:]]
ang=ang[0,oo[0][:]]
angrad=ang*Pi/180.

cosrdg=np.cos( angrad )
sinrdg=np.sin( angrad )

fig=plt.figure( figsize =(20,9))
ax=fig.add_subplot(2,2,1)
po=ax.plot( mxd, '.')
po=ax.set_title( 'MXDIS' ,fontsize=12, loc='center')
ax=fig.add_subplot(2,2,2)
po=ax.plot( cln, '.')
po=ax.set_title( 'CLNGT' ,fontsize=12, loc='center')
ax=fig.add_subplot(2,2,3)
po=ax.plot( ang,cosrdg, '.')
po=ax.set_title( 'Cos(ANGLL)' ,fontsize=12, loc='center')
ax=fig.add_subplot(2,2,4)
po=ax.plot( ang,sinrdg, '.')
po=ax.set_title( 'Sin (ANGLL)' ,fontsize=12, loc='center')

In [ ]:
# Numpy arrays are [nT,nL,ns]
# Select columns with MXD>h_crit nS_0=>nS
# and restrict to bottom 10-layers
# Transpose to [nL,nt,ns_crit]
# Then reshape to [nL,nT*ns_crit]
u=U.values
u=u[:,21:,oo[0][:]]
u=np.transpose(u, (1,0,2) )
nL,nT,nS=np.shape(u)
u=np.reshape( u, (nL,nT*nS ) )/10.

v=V.values
v=v[:,21:,oo[0][:]]
v=np.transpose(v, (1,0,2) )
v=np.reshape( v, (nL,nT*nS ) )/10.

te=T.values
te=te[:,21:,oo[0][:]]
te=np.transpose(te, (1,0,2) )
te=np.reshape( te, (nL,nT*nS ) )/10.


In [ ]:
utn=Utn.values
print(np.shape(utn))
plt.plot(utn[100:500,30,:].flatten())

In [ ]:
# Numpy arrays are [nT,nL,ns]
# Select columns with MXD>h_crit ns=>ns_crit
# and restrict to bottom 10-layers
# Transpose to [nL,nt,ns_crit]
# Then reshape to [nL,nT*ns_crit]
utn=Utn.values
utn=utn[:,21:,oo[0][:]]
utn=np.transpose(utn, (1,0,2) )
utn=np.reshape( utn, (nL,nT*nS ) )/10.

vtn=Vtn.values
vtn=vtn[:,21:,oo[0][:]]
vtn=np.transpose(vtn, (1,0,2) )
vtn=np.reshape( vtn, (nL,nT*nS ) )/10.




In [ ]:
mxdx=np.zeros([1,nT,nS])
clnx=np.zeros([1,nT,nS])
cosrx=np.zeros([1,nT,nS])
sinrx=np.zeros([1,nT,nS])
for iT in np.arange(nT):
    mxdx[0,iT,:]=mxd
    clnx[0,iT,:]=cln
    cosrx[0,iT,:]=cosrdg
    sinrx[0,iT,:]=sinrdg
mxdx = np.reshape( mxdx, (1,nT*nS ) )/1000.
clnx = np.reshape( clnx, (1,nT*nS ) )/100.

In [ ]:
print(np.shape(mxdx))
plt.plot(cln.view())

In [ ]:
A=np.r_[u,v,mxdx,clnx]
B=np.r_[utn,vtn]


In [ ]:
print(np.shape(A))


In [ ]:
A=A[:,0:20000]
B=B[:,0:20000]
A=np.transpose(A)
B=np.transpose(B)

print(np.shape(A))


In [ ]:
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
forest_reg.fit(A, B)

In [ ]:
B2=forest_reg.predict(A)

In [ ]:
plt.scatter(B2.flatten(),B.flatten())

In [ ]:
AA=np.r_[u,v,mxdx,clnx]
BB=np.r_[utn,vtn]
AA=np.transpose(AA)
BB=np.transpose(BB)
print(np.shape(BB))


In [ ]:
A_val=AA[60000:100000,:]
B_val=BB[60000:100000,:]


In [ ]:
B_pred=forest_reg.predict(A_val)

In [ ]:
print(np.shape(B_pred),np.shape(B_val))

In [ ]:
plt.scatter(B_pred.flatten(),B_val.flatten())